In [75]:
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "0"
from N2_dataprep_time import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, TimeDistributed, Conv1D, MaxPooling1D
from random import shuffle, random

In [76]:
epochs = 6
frame_in = 48
frame_out = 2
test_ratio = 0.2

In [77]:
class CNNLSTM(Model):
    def __init__(self):
        super().__init__()
        
    def setup(self, frame_in, frame_out, filters_=24, kernel_size_=5, pool_size_=4):
        self.model = Sequential()
        self.model.add(TimeDistributed(Conv1D(filters=filters_, kernel_size=kernel_size_, activation='relu'), input_shape=(None, frame_in, 1)))
        self.model.add(TimeDistributed(MaxPooling1D(pool_size=pool_size_)))
        self.model.add(TimeDistributed(Flatten()))
        self.model.add(LSTM(24, activation='relu'))
        self.model.add(Dense(frame_out))
        self.model.compile(optimizer='adam', loss='mse')

In [78]:
str_fn = '-'+str(frame_in)+'-'+str(frame_out)
try:
    train_inp = pk.load(open("temp/trinp"+str_fn, 'rb'))
    train_oup = pk.load(open("temp/troup"+str_fn, 'rb'))
    test_inp = pk.load(open("temp/teinp"+str_fn, 'rb'))
    test_oup = pk.load(open("temp/teoup"+str_fn, 'rb'))
except:
    train_inp, train_oup, test_inp, test_oup = compound(setup(test_ratio, frame_in, frame_out))
    pk.dump(train_inp, open("temp/trinp"+str_fn, 'wb'))
    pk.dump(train_oup, open("temp/troup"+str_fn, 'wb'))
    pk.dump(test_inp, open("temp/teinp"+str_fn, 'wb'))
    pk.dump(test_oup, open("temp/teoup"+str_fn, 'wb'))

In [79]:
tr_divisor = 1
te_divisor = 25

train_limit = len(train_inp) // tr_divisor
test_limit = len(test_inp) // te_divisor

train_limited_inp = train_inp[:train_limit]
train_limited_oup = train_oup[:train_limit]
test_limited_inp = test_inp[:test_limit]
test_limited_oup = test_oup[:test_limit]

c = CNNLSTM()

In [80]:
session_num = getSession()
def globalTrain(train_inp_, train_oup_, model_, frame_in_, frame_out_, epochs__):
    model_.setup(frame_in_, frame_out_)
    model_.setData(train_inp_, train_oup_)
    model_.train('Global', session_num, epochs_=epochs__, verbose_=2)
    #model_.summarize()
    return model_

In [81]:
#model = globalTrain(train_limited_inp, train_limited_oup, c, frame_in, frame_out, 1)

In [82]:
def testCase(test_inp_, test_oup_, model_, frame_in_, frame_out_, epochs__, label_):
    p = Prediction(test_inp_, test_oup_, model_, frame_in_, frame_out_, session_num)
    p.predict()
    p.summary(label=label_, epochs_=epochs__)

#testCase(test_limited_inp, test_limited_oup, c, frame_in, frame_out, epochs, 'Global')

In [83]:
def truncateSet(sets):
    truncated = []
    for s in sets:
        truncated.append([s[2],s[3]])
    return truncated

In [84]:
def loadSets(test_ratio_, frame_in_, frame_out_, str_fn_):
    try:
        sets = pk.load(open("temp/truncset"+str(test_ratio_)+str_fn_, 'rb'))
    except:
        sets = truncateSet(setup(test_ratio_, frame_in_, frame_out_))
        pk.dump(sets, open("temp/truncset"+str(test_ratio_)+str_fn_, 'wb'))
    return sets

sets = loadSets(test_ratio, frame_in, frame_out, str_fn)

In [85]:
def localTest(sets, frame_in_, frame_out_, model_, te_divisor_, epochs__=6):
    sindex = 0
    for s in sets:
        cases = len(s[0]) // te_divisor_
        if sindex in [3, 5, 10]:
            testCase(s[0][:cases], s[1][:cases], model_, frame_in_, frame_out_, epochs__, str(sindex))
        sindex += 1

#localTest(sets, frame_in, frame_out, c, te_divisor)

In [86]:
def printNoData(stats):
    for s in stats:
        print("{:02d} | TR {:5d} | DEM {:8.2f} | WEI {:5.0f} | POR {:0.5f}".format(s['id'], s['train'], s['dem'], s['weight'], s['portion']))

def makeStats(sets_):
    # Make statistics
    ss = deepcopy(sets_)
    statistics = [{'id': s + 1, 'data' : ss[s], 'train': 0, 'dem': 0, 'weight': 0, 'portion' : 0} for s in range(len(ss))]

    for s in range(len(ss)):
        statistics[s]['train'] = len(ss[s][1])
        s_dem = 0
        for i in range(len(ss[s][1])):
            s_dem += ss[s][1][i][0]
        statistics[s]['dem'] = s_dem
        statistics[s]['weight'] = (len(ss[s][1]) ** 0.5) * s_dem / 100

    total_train = 0
    total_dem = 0
    total_weight = 0

    for s in range(len(statistics)):
        total_train += statistics[s]['train']
        total_dem += statistics[s]['train']
        total_weight += statistics[s]['weight']
        
    for s in range(len(statistics)):
        statistics[s]['portion'] = statistics[s]['weight']/total_weight

    printNoData(statistics)
    return statistics, total_weight

sets_full = loadSets(1, frame_in, frame_out, str_fn)
stats = makeStats(sets_full)


01 | TR 195694 | DEM 13875.48 | WEI 61381 | POR 0.05886
02 | TR 195694 | DEM 19245.53 | WEI 85137 | POR 0.08164
03 | TR 48886 | DEM  3285.88 | WEI  7265 | POR 0.00697
04 | TR 198664 | DEM 110302.38 | WEI 491637 | POR 0.47145
05 | TR 48884 | DEM  1841.67 | WEI  4072 | POR 0.00390
06 | TR 48852 | DEM 13908.35 | WEI 30741 | POR 0.02948
07 | TR 48820 | DEM 26139.62 | WEI 57756 | POR 0.05539
08 | TR 97764 | DEM 34313.54 | WEI 107289 | POR 0.10288
09 | TR 97098 | DEM  3098.26 | WEI  9654 | POR 0.00926
10 | TR 48886 | DEM 16082.98 | WEI 35560 | POR 0.03410
11 | TR 198543 | DEM   918.53 | WEI  4093 | POR 0.00392
12 | TR 195694 | DEM 24636.05 | WEI 108983 | POR 0.10451
13 | TR 48769 | DEM  7381.96 | WEI 16302 | POR 0.01563
14 | TR 48886 | DEM  2769.05 | WEI  6122 | POR 0.00587
15 | TR 48837 | DEM  7610.32 | WEI 16818 | POR 0.01613


In [87]:
# Poisoning factor
poisoning_factors = [0.101, 0.255, 0.412, 0.521, 0.635, 0.762]

def poison(stats_, defect_ratio):
    seth_ = deepcopy(stats_)
    seth_, weight = seth_[0], seth_[1]
    shuffle(seth_)
    printNoData(seth_)
    altered = 0
    for i in range(len(seth_)):
        for l in range(len(seth_[i]['data'])):
            for j in range(len(seth_[i]['data'][l])):
                for k in range(len(seth_[i]['data'][l][j])):
                    if defect_ratio < 0:
                        break
                    seth_[i]['data'][l][j][k] = seth_[i]['data'][l][j][k] * (random() * 1 - 0.5)
                    altered += 1/2
                    defect_ratio -= (0.5 / weight)
    print("Altered: " + str(int(altered)), defect_ratio)
    return seth_

set_poisoned = poison(stats, poisoning_factors[0])

08 | TR 97764 | DEM 34313.54 | WEI 107289 | POR 0.10288
04 | TR 198664 | DEM 110302.38 | WEI 491637 | POR 0.47145
09 | TR 97098 | DEM  3098.26 | WEI  9654 | POR 0.00926
07 | TR 48820 | DEM 26139.62 | WEI 57756 | POR 0.05539
15 | TR 48837 | DEM  7610.32 | WEI 16818 | POR 0.01613
14 | TR 48886 | DEM  2769.05 | WEI  6122 | POR 0.00587
01 | TR 195694 | DEM 13875.48 | WEI 61381 | POR 0.05886
11 | TR 198543 | DEM   918.53 | WEI  4093 | POR 0.00392
12 | TR 195694 | DEM 24636.05 | WEI 108983 | POR 0.10451
13 | TR 48769 | DEM  7381.96 | WEI 16302 | POR 0.01563
02 | TR 195694 | DEM 19245.53 | WEI 85137 | POR 0.08164
03 | TR 48886 | DEM  3285.88 | WEI  7265 | POR 0.00697
06 | TR 48852 | DEM 13908.35 | WEI 30741 | POR 0.02948
05 | TR 48884 | DEM  1841.67 | WEI  4072 | POR 0.00390
10 | TR 48886 | DEM 16082.98 | WEI 35560 | POR 0.03410
Altered: 105324 -6.785084640251388e-08


In [88]:
def uniteStats(stats_):
    train_inp_ = []
    train_oup_ = []
    for s in stats_:
        train_inp_.extend(s['data'][0])
        train_oup_.extend(s['data'][1])
        pass
    return arr(train_inp_), arr(train_oup_)

modp = CNNLSTM()
def trainPoisoned(test_pin, test_pou, model_, frame_in_, frame_out_, epochs__):
    model_.setup(frame_in_, frame_out_)
    model_.setData(test_pin, test_pou)
    model_.train('GloPoi', getSession(), verbose_=2, epochs_=epochs__)
    return model_

t1, t2 = uniteStats(set_poisoned)

In [89]:
mod_po = trainPoisoned(t1, t2, modp, frame_in, frame_out, epochs)

Epoch 1/6
49062/49062 - 91s - loss: 0.0020
Epoch 2/6
49062/49062 - 100s - loss: 0.0015
Epoch 3/6
49062/49062 - 106s - loss: 0.0014
Epoch 4/6
49062/49062 - 102s - loss: 0.0013
Epoch 5/6
49062/49062 - 98s - loss: 0.0013
Epoch 6/6
49062/49062 - 114s - loss: 0.0013


In [90]:
testCase(test_limited_inp, test_limited_oup, mod_po, frame_in, frame_out, epochs, 'Global-Poi')

12559 | ||||||||||254 | L Global-Poi | TD 890.06377 | TC 12559 | AD 0.07 | [16.]% | Dur 547.6679725646973


In [91]:
localTest(sets, frame_in, frame_out, mod_po, te_divisor)

1589 | ||||||||||254 | L 3 | TD 878.46152 | TC 1589 | AD 0.55 | [1.39]% | Dur 66.78307032585144
390 | ||||||||||254 | L 5 | TD 114.69524 | TC 390 | AD 0.29 | [3.81]% | Dur 15.191904544830322
1588 | ||||||||||254 | L 10 | TD 7.59838 | TC 1588 | AD 0.0 | [60.82]% | Dur 71.50827884674072
